In [1]:
text = """
Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему. Все смешалось в доме Облонских. Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме. Положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами, и всеми членами семьи, и домочадцами. Все члены семьи и домочадцы чувствовали, что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой, чем они, члены семьи и домочадцы Облонских. Жена не выходила из своих комнат, мужа третий день не было дома. Дети бегали по всему дому, как потерянные; англичанка поссорилась с экономкой и написала записку приятельнице, прося приискать ей новое место; повар ушел вчера со двора, во время самого обеда; черная кухарка и кучер просили расчета.
"""

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from razdel import sentenize
import numpy as np
import pandas as pd
from tqdm import tqdm
from termcolor import colored

In [3]:
model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

H:\Anaconda\envs\transformers\lib\site-packages\transformers\configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [4]:
def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    label_texts
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba

In [5]:
classes_0 = ['Правда', 'Истина', 'Действительность', 'Реальность', 'Факт']
classes_1 = ['Ложь', 'Неправда', 'Обман', 'Заблуждение', 'Выдумка']
classes_2 = ['Изложение', 'Повествование', 'Информирование', 'Описание', 'Исследование']
classes_3 = ['Восхваление', 'Запугивание', 'Нагнетание', 'Расхваливание', 'Пропаганда']
classes_list = [classes_0, classes_1, classes_2, classes_3]

In [6]:
df_classes = pd.DataFrame(columns=['Слово', 'Класс'])
for i, e in enumerate(classes_list):
    df_e = pd.DataFrame(e, columns=['Слово'])
    df_e['Класс'] = i
    df_classes = pd.concat([df_classes, df_e])
df_classes = df_classes.reset_index(drop=True)

In [7]:
sentences = [sent.text for sent in sentenize(text)]

In [8]:
classes = df_classes.loc[:,'Слово'].to_list()
results = []
for sentence in tqdm(sentences):
    estimates = predict_zero_shot(sentence, classes, model, tokenizer)
    results.append(estimates)

df_estimates = pd.DataFrame()
df_estimates['Предложение'] = sentences
df_estimates = df_estimates.join(pd.DataFrame(results, columns=classes))

100%|██████████| 7/7 [00:38<00:00,  5.47s/it]


In [9]:
estimates = ['Правда', 'Ложь', 'Строгость', 'Эмоциональность']
df_final_estimates = pd.DataFrame()
df_final_estimates['Предложение'] = df_estimates['Предложение']
for i, c in enumerate(classes_list):
    df_final_estimates[estimates[i]] = df_estimates.apply(lambda row: row[c].sum(),axis=1)

In [10]:
columns = ['Предложение', 'Следствие', 'Противоречие']
df_logic = pd.DataFrame(columns=columns)

for sentence in tqdm(sentences):
    prev_text = " ".join(sentences[:sentences.index(sentence)])
    with torch.inference_mode():
        out = model(**tokenizer(prev_text, sentence, return_tensors='pt', truncation=True).to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
    prev_analysis = {v: proba[k] for k, v in model.config.id2label.items()}

    post_text = " ".join(sentences[sentences.index(sentence)+1:])
    with torch.inference_mode():
        out = model(**tokenizer(sentence, post_text, return_tensors='pt', truncation=True).to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
    post_analysis = {v: proba[k] for k, v in model.config.id2label.items()}
    
    new_df = pd.DataFrame([
        [sentence,
         0.5*(prev_analysis['entailment']+post_analysis['entailment']), 
         0.5*(prev_analysis['contradiction']+post_analysis['contradiction'])
        ]], columns = columns)
    df_logic = df_logic.append(new_df)

df_logic = df_logic.reset_index(drop=True)

100%|██████████| 7/7 [00:14<00:00,  2.08s/it]


In [11]:
df_results = df_final_estimates.merge(df_logic, on='Предложение')

In [12]:
df_results['Правда+'] = df_results['Правда']*(1+df_results['Следствие'])*(1-df_results['Противоречие'])
df_results['Ложь+'] = df_results['Ложь']*(1+df_results['Следствие'])*(1-df_results['Противоречие'])

In [13]:
df_results['Мат.ожидание Правда'] = df_results['Правда+'] - df_results['Ложь+']
df_results['Мат.ожидание Строгость'] = df_results['Строгость'] - df_results['Эмоциональность']
df_results['Знак МО Правда'] = np.sign(df_results['Мат.ожидание Правда'])
df_results['Знак МО Строгость'] = np.sign(df_results['Мат.ожидание Строгость'])
df_results['Качество'] = np.sign(df_results['Знак МО Правда'] + df_results['Знак МО Строгость']).astype(int)

In [14]:
colors = {-1:'red', 0:'yellow', 1:'green'}
for index, row in df_results.iterrows():
    print(colored(row['Предложение'], colors[row['Качество']]))

Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.
Все смешалось в доме Облонских.
Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме.
Положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами, и всеми членами семьи, и домочадцами.
Все члены семьи и домочадцы чувствовали, что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой, чем они, члены семьи и домочадцы Облонских.
Жена не выходила из своих комнат, мужа третий день не было дома.
Дети бегали по всему дому, как потерянные; англичанка поссорилась с экономкой и написала записку приятельнице, прося приискать ей новое место; повар ушел вчера со двора, во время самого обеда; черная кухарка и кучер просили расчета.
